<a href="https://colab.research.google.com/github/Abdulla-creator-coder/Test-project/blob/main/Bankcreditrisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
""" The constraint faced in this project is only 4% of 24000(approx) values are deffaulters , hence model had difficulty in understanding the features of defaulters."""
"""The next constraint is only 24000 values which is not large enough to train the models properly"""
"""The added up constraint in this project is the lack of domain knowledge"""

""Logistic regresion without SMOTE GINI=0.25,
Logistic regression with SMOTE GINI=0.26
Logistic regression with WOE train Gini=0.24
Logistic regression with WOE test GINI=0.26
Random Forest Train Gini: 0.5839
Random Forest Test Gini : 0.3092
Gradient Boosting Train Gini: 0.6867
Gradient Boosting Test Gini : 0.2807""

In [1]:
pip install mysql-connector-python ##connecting code

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 20.5 MB/s eta 0:00:00


In [2]:
import mysql.connector
import pandas as pd

In [3]:
conn = mysql.connector.connect(
    host="18.136.157.135",
    user="dm_team1",
    password="DM!$Team&279@20!",
    database="project_banking"
)
                                                    ##connecting code to database
print("Connected successfully")
#"SHOW DATABASES;"

Connected successfully


In [4]:
cursor = conn.cursor()
cursor.execute("SHOW DATABASES;")       ##checking code to access info file
print(cursor.fetchall())

[('information_schema',), ('project_banking',)]


In [5]:
query_account = "SELECT * FROM Cust_Account"  ##connecting code to first data file
df_account = pd.read_sql(query_account, conn)

print(df_account.shape)
df_account.head()

/tmp/ipython-input-271322330.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_account = pd.read_sql(query_account, conn)


(186329, 21)


,dt_opened,customer_no,upload_dt,acct_type,owner_indic,opened_dt,last_paymt_dt,closed_dt,reporting_dt,high_credit_amt,...,amt_past_due,paymenthistory1,paymenthistory2,paymt_str_dt,paymt_end_dt,creditlimit,cashlimit,rateofinterest,paymentfrequency,actualpaymentamount
0,10-Nov-15,12265,20-Oct-15,6,1,09-Jun-13,30-Jun-14,05-Jul-14,30-Sep-15,20900,...,,"""""""STDSTDSTDXXXXXXXXXXXXXXXSTDXXXXXXXXXXXXXXXS...",,01-Sep-15,01-Jul-14,,,,,
1,10-Nov-15,12265,20-Oct-15,10,1,25-May-12,06-Sep-15,,03-Oct-15,16201,...,,"""""""0000000000000000000000000000000000000000000...","""""""000000000000000000000000000XXX0000000000000...",01-Oct-15,01-Nov-12,14000,1400,,3,5603
2,10-Nov-15,12265,20-Oct-15,10,1,22-Mar-12,31-Aug-15,,30-Sep-15,41028,...,,"""""""0000000000000000000000000000000000000000000...","""""""0000000000000000000000000000000000000000000...",01-Sep-15,01-Oct-12,,,,,
3,20-Jul-15,15606,09-Jul-15,10,1,13-Jan-06,,26-Jul-07,31-Jan-09,93473,...,,"""""""1200900600600600300000000000000000000000000...",,01-Jul-07,01-Feb-06,,,,,
4,20-Jul-15,15606,09-Jul-15,6,1,18-Jan-15,05-May-15,,31-May-15,20250,...,,"""""""000000000000000""""""",,01-May-15,01-Jan-15,,,,,


In [6]:
import pandas as pd
import numpy as np
import re

def clean_ph(x):         ##function to clean payment history clean
    if pd.isna(x):
        return None
    x = str(x)
    x = x.replace('"', '').replace(',', '').strip()
    x = x.replace(' ', '')
    return x if x != '' else None

df_account['ph1_clean'] = df_account['paymenthistory1'].apply(clean_ph)
df_account['ph2_clean'] = df_account['paymenthistory2'].apply(clean_ph)

In [7]:
def detect_ph_type(x):
    if x is None:
        return 'missing'
    if re.fullmatch(r'[A-Z]+', x):     ##seperating out text and numerical columns
        return 'alpha'
    if re.fullmatch(r'\d+', x):
        return 'numeric'
    return 'mixed'

df_account['ph1_type'] = df_account['ph1_clean'].apply(detect_ph_type)
df_account['ph2_type'] = df_account['ph2_clean'].apply(detect_ph_type)

In [8]:
def parse_alpha_ph(x):
    if x is None:        ## function for parsing alphabetical characters
        return []
    return [x[i:i+3] for i in range(0, len(x), 3)]

In [9]:
def parse_numeric_ph(x):
    if x is None:          ##function for parsing numerical characters
        return []
    x = ''.join(c for c in x if c.isdigit())
    return [int(x[i:i+3]) for i in range(0, len(x), 3)]

In [10]:
def parse_ph(row, col_clean, col_type):
    if row[col_type] == 'alpha':
        return parse_alpha_ph(row[col_clean])
    elif row[col_type] == 'numeric':              ##code for applying parsing into the respective columns
        return parse_numeric_ph(row[col_clean])
    else:
        return []

df_account['ph1_parsed'] = df_account.apply(
    lambda row: parse_ph(row, 'ph1_clean', 'ph1_type'),
    axis=1
)

df_account['ph2_parsed'] = df_account.apply(
    lambda row: parse_ph(row, 'ph2_clean', 'ph2_type'),
    axis=1
)

In [11]:
def convert_alpha_to_dpd(lst):
    dpd = []
    for val in lst:
        if val in ['STD', 'XXX']:
            dpd.append(0)                                     ##code for converting alphabetical characters into appropriate numerical character
        else:
            dpd.append(30)
    return dpd

df_account['ph1_dpd'] = df_account.apply(
    lambda row: convert_alpha_to_dpd(row['ph1_parsed'])
    if row['ph1_type'] == 'alpha'
    else row['ph1_parsed'],
    axis=1
)

df_account['ph2_dpd'] = df_account.apply(
    lambda row: convert_alpha_to_dpd(row['ph2_parsed'])
    if row['ph2_type'] == 'alpha'
    else row['ph2_parsed'],
    axis=1
)

In [12]:
def create_features(lst):
    if not lst:
        return pd.Series([0, 0, 0, 0])

    length = len(lst)
    max_dpd = max(lst)
    has_30_plus = int(any(x >= 30 for x in lst))           ##function for categorisation based on payment history
    good_months = sum(1 for x in lst if x == 0)

    return pd.Series([length, max_dpd, has_30_plus, good_months])

In [17]:
df_account[['ph1_length', 'ph1_max_dpd',
            'ph1_has_30_plus', 'ph1_good_months']] = \
df_account['ph1_dpd'].apply(create_features).apply(pd.Series)

In [19]:
df_account[['ph2_length', 'ph2_max_dpd',
            'ph2_has_30_plus', 'ph2_good_months']] = \
df_account['ph2_dpd'].apply(create_features).apply(pd.Series)

In [20]:
df_account['ph1_perc_good'] = np.where(
    df_account['ph1_length'] > 0,
    df_account['ph1_good_months'] / df_account['ph1_length'],
    0
)                                                                                     ##creating features for model improvement

df_account['ph2_perc_good'] = np.where(
    df_account['ph2_length'] > 0,
    df_account['ph2_good_months'] / df_account['ph2_length'],
    0
)

In [21]:
drop_cols = [
    'paymenthistory1', 'paymenthistory2',
    'ph1_clean', 'ph2_clean',
    'ph1_type', 'ph2_type',                                                        ##dropping unwanted and Variance influencing columns
    'ph1_parsed', 'ph2_parsed',
    'ph1_dpd', 'ph2_dpd'
]

df_account = df_account.drop(columns=drop_cols)

In [22]:
account_date_columns=['dt_opened','upload_dt','opened_dt','last_paymt_dt','closed_dt', 'reporting_dt','paymt_str_dt', 'paymt_end_dt']
account_numeric_columns=['customer_no', 'acct_type','owner_indic','high_credit_amt', 'creditlimit','cashlimit','rateofinterest','paymentfrequency','actualpaymentamount','cur_balance_amt', 'amt_past_due']
account_payment_columns=['paymenthistory1','paymenthistory2']

In [23]:
for col in account_numeric_columns:
    df_account[col] = pd.to_numeric(
        df_account[col],                               ##converting from object to numeric type data
        errors='coerce'
    )

In [24]:
for col in account_date_columns:
    df_account[col] = pd.to_datetime(                   ##converting from object to date type data
        df_account[col],
        errors='coerce'
    )

/tmp/ipython-input-1576281799.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_account[col] = pd.to_datetime(                   ##converting from object to date type data
/tmp/ipython-input-1576281799.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_account[col] = pd.to_datetime(                   ##converting from object to date type data
/tmp/ipython-input-1576281799.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_account[col] = pd.to_datetime(                   ##converting from object to date type data
/tmp/ipython-input-1576281799.py:2: UserWarning: 

In [25]:
df_account.columns

Index(['dt_opened', 'customer_no', 'upload_dt', 'acct_type', 'owner_indic',
       'opened_dt', 'last_paymt_dt', 'closed_dt', 'reporting_dt',
       'high_credit_amt', 'cur_balance_amt', 'amt_past_due', 'paymt_str_dt',
       'paymt_end_dt', 'creditlimit', 'cashlimit', 'rateofinterest',
       'paymentfrequency', 'actualpaymentamount', 'ph1_length', 'ph1_max_dpd',
       'ph1_has_30_plus', 'ph1_good_months', 'ph2_length', 'ph2_max_dpd',
       'ph2_has_30_plus', 'ph2_good_months', 'ph1_perc_good', 'ph2_perc_good'],
      dtype='object')

In [26]:
len(df_account.columns)

29

In [27]:
a=len(account_date_columns)
b=len(account_numeric_columns)
c=len(account_payment_columns)
print(a+b+c)

21


In [28]:
df_account.dtypes

,0
dt_opened,datetime64[ns]
customer_no,int64
upload_dt,datetime64[ns]
acct_type,int64
owner_indic,int64
opened_dt,datetime64[ns]
last_paymt_dt,datetime64[ns]
closed_dt,datetime64[ns]
reporting_dt,datetime64[ns]
high_credit_amt,float64


In [29]:
df_account.isna().sum().sort_values(ascending=False).head(10)   ##checking of null values



,0
amt_past_due,185453
rateofinterest,161506
cashlimit,151047
actualpaymentamount,145276
creditlimit,137477
paymentfrequency,122436
closed_dt,109075
last_paymt_dt,25487
high_credit_amt,8875
opened_dt,455


In [30]:
df_account.dtypes

,0
dt_opened,datetime64[ns]
customer_no,int64
upload_dt,datetime64[ns]
acct_type,int64
owner_indic,int64
opened_dt,datetime64[ns]
last_paymt_dt,datetime64[ns]
closed_dt,datetime64[ns]
reporting_dt,datetime64[ns]
high_credit_amt,float64


In [31]:
df_account.shape
df_account.dtypes.value_counts()

,count
int64,12
float64,9
datetime64[ns],8


In [32]:
df_account.head()

,dt_opened,customer_no,upload_dt,acct_type,owner_indic,opened_dt,last_paymt_dt,closed_dt,reporting_dt,high_credit_amt,...,ph1_length,ph1_max_dpd,ph1_has_30_plus,ph1_good_months,ph2_length,ph2_max_dpd,ph2_has_30_plus,ph2_good_months,ph1_perc_good,ph2_perc_good
0,2015-11-10,12265,2015-10-20,6,1,2013-06-09,2014-06-30,2014-07-05,2015-09-30,20900.0,...,15,0,0,15,0,0,0,0,1.000000,0.0
1,2015-11-10,12265,2015-10-20,10,1,2012-05-25,2015-09-06,NaT,2015-10-03,16201.0,...,18,0,0,18,0,0,0,0,1.000000,0.0
2,2015-11-10,12265,2015-10-20,10,1,2012-03-22,2015-08-31,NaT,2015-09-30,41028.0,...,18,0,0,18,18,0,0,18,1.000000,1.0
3,2015-07-20,15606,2015-07-09,10,1,2006-01-13,NaT,2007-07-26,2009-01-31,93473.0,...,18,120,1,12,0,0,0,0,0.666667,0.0
4,2015-07-20,15606,2015-07-09,6,1,2015-01-18,2015-05-05,NaT,2015-05-31,20250.0,...,5,0,0,5,0,0,0,0,1.000000,0.0


In [33]:
df_account.columns

Index(['dt_opened', 'customer_no', 'upload_dt', 'acct_type', 'owner_indic',
       'opened_dt', 'last_paymt_dt', 'closed_dt', 'reporting_dt',
       'high_credit_amt', 'cur_balance_amt', 'amt_past_due', 'paymt_str_dt',
       'paymt_end_dt', 'creditlimit', 'cashlimit', 'rateofinterest',
       'paymentfrequency', 'actualpaymentamount', 'ph1_length', 'ph1_max_dpd',
       'ph1_has_30_plus', 'ph1_good_months', 'ph2_length', 'ph2_max_dpd',
       'ph2_has_30_plus', 'ph2_good_months', 'ph1_perc_good', 'ph2_perc_good'],
      dtype='object')

In [34]:
corr_matrix = df_account.corr(numeric_only=True)

corr_abs = corr_matrix.abs()                                 ##syntax for identifying and removing high corelation features

upper_triangle = corr_abs.where(
    np.triu(np.ones(corr_abs.shape), k=1).astype(bool)
)

threshold = 0.8

high_corr_pairs = (
    upper_triangle.stack()
    .reset_index()
)

high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']
high_corr_pairs = high_corr_pairs[
    high_corr_pairs['Correlation'] > threshold
].sort_values(by='Correlation', ascending=False)

high_corr_pairs

,Feature_1,Feature_2,Correlation
195,ph2_length,ph2_good_months,0.985021
165,ph1_length,ph1_good_months,0.980169
206,ph2_good_months,ph2_perc_good,0.856732
197,ph2_length,ph2_perc_good,0.841153


In [35]:
def drop_highly_correlated(df, threshold=0.8):

    corr_matrix = df.corr(numeric_only=True).abs()

    upper = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )

    to_drop = [
        column for column in upper.columns
        if any(upper[column] > threshold)
    ]
    df_reduced = df.drop(columns=to_drop)

    return df_reduced, to_drop

In [36]:
df_reduced, dropped_columns = drop_highly_correlated(df_account, threshold=0.8)

print("Dropped Columns:")
print(dropped_columns)

Dropped Columns:
['ph1_good_months', 'ph2_good_months', 'ph2_perc_good']


In [37]:
df_account.dtypes

,0
dt_opened,datetime64[ns]
customer_no,int64
upload_dt,datetime64[ns]
acct_type,int64
owner_indic,int64
opened_dt,datetime64[ns]
last_paymt_dt,datetime64[ns]
closed_dt,datetime64[ns]
reporting_dt,datetime64[ns]
high_credit_amt,float64


In [38]:
df_account.duplicated().sum()    ##checking of duplicate rows

np.int64(2410)

In [39]:
df_account = df_account.drop_duplicates()    ##dropping of duplicate rows

In [40]:
df_account_agg = df_account.groupby('customer_no').agg({
   ##code for aggregating inorder to create one row per customer number
    #  Monetary – Total exposure
    'high_credit_amt': 'sum',
    'cur_balance_amt': 'sum',
    'amt_past_due': 'sum',
    'creditlimit': 'sum',
    'cashlimit': 'sum',

    #  Average behaviour
    'rateofinterest': 'mean',
    'actualpaymentamount': 'mean',
    'paymentfrequency': 'mean',

    #  Risk indicators
    'ph1_max_dpd': 'max',
    'ph2_max_dpd': 'max', 'ph1_has_30_plus': 'max',
    'ph2_has_30_plus': 'max',

    #  Payment behaviour
    'ph1_perc_good': 'mean',
    'ph2_perc_good': 'mean',
    'ph1_good_months': 'sum',
    'ph2_good_months': 'sum',

    #  Dates
    'opened_dt': 'min',          # first account opened
    'closed_dt': 'max',          # last closed date
    'last_paymt_dt': 'max',      # most recent payment
    'reporting_dt': 'max',
    'paymt_str_dt': 'min',
    'paymt_end_dt': 'max',

    #  Account count
    'acct_type': 'count'

}).reset_index()

In [41]:
query_enquiry = "SELECT * FROM Cust_Enquiry"
df_enquiry = pd.read_sql(query_enquiry, conn)   ##collecting data from database for 2nd datasheet

print(df_enquiry.shape)
df_enquiry.head()

/tmp/ipython-input-968898736.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_enquiry = pd.read_sql(query_enquiry, conn)   ##collecting data from database for 2nd datasheet


(413188, 6)


,dt_opened,customer_no,upload_dt,enquiry_dt,enq_purpose,enq_amt
0,18-Apr-15,1,21-Apr-15,19-Dec-14,2,3500000
1,18-Apr-15,1,21-Apr-15,05-Mar-14,5,500000
2,18-Apr-15,1,21-Apr-15,05-Mar-14,0,50000
3,18-Apr-15,1,21-Apr-15,22-Feb-14,10,50000
4,18-Apr-15,1,21-Apr-15,11-Jun-13,10,1000


In [42]:
df_enquiry.head(5)

,dt_opened,customer_no,upload_dt,enquiry_dt,enq_purpose,enq_amt
0,18-Apr-15,1,21-Apr-15,19-Dec-14,2,3500000
1,18-Apr-15,1,21-Apr-15,05-Mar-14,5,500000
2,18-Apr-15,1,21-Apr-15,05-Mar-14,0,50000
3,18-Apr-15,1,21-Apr-15,22-Feb-14,10,50000
4,18-Apr-15,1,21-Apr-15,11-Jun-13,10,1000


In [43]:
df_enquiry.dtypes

,0
dt_opened,object
customer_no,object
upload_dt,object
enquiry_dt,object
enq_purpose,object
enq_amt,object


In [44]:
df_enquiry.columns

Index(['dt_opened', 'customer_no', 'upload_dt', 'enquiry_dt', 'enq_purpose',
       'enq_amt'],
      dtype='object')

In [45]:
enquiry_date_columns=['dt_opened','upload_dt','enquiry_dt']
enquiry_num_columns=['customer_no','enq_purpose','enq_amt']

In [46]:
for col in enquiry_date_columns:
    df_enquiry[col] = pd.to_datetime(df_enquiry[col], errors='coerce', dayfirst=True)  ##converting from object to data type columns

/tmp/ipython-input-4123485129.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_enquiry[col] = pd.to_datetime(df_enquiry[col], errors='coerce', dayfirst=True)  ##converting from object to data type columns
/tmp/ipython-input-4123485129.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_enquiry[col] = pd.to_datetime(df_enquiry[col], errors='coerce', dayfirst=True)  ##converting from object to data type columns
/tmp/ipython-input-4123485129.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_enquiry[col] = pd.to_datetime(df_enquiry[col], errors='coerce', dayfirst=

In [47]:
for col in enquiry_num_columns:
    df_enquiry[col] = pd.to_numeric(df_enquiry[col], errors='coerce')   ##converting from object to numeric type columns


In [48]:
df_enquiry.dtypes

,0
dt_opened,datetime64[ns]
customer_no,int64
upload_dt,datetime64[ns]
enquiry_dt,datetime64[ns]
enq_purpose,float64
enq_amt,float64


In [49]:
df_enquiry.duplicated().sum()

np.int64(13827)

In [50]:
df_enquiry = df_enquiry.drop_duplicates()

In [51]:
df_enquiry_agg = df_enquiry.groupby('customer_no').agg({

    #  Total enquiry amount
    'enq_amt': ['sum', 'mean', 'max'],

    #  Date based features
    'enquiry_dt': ['min', 'max', 'count'],

    # Optional (if purpose is meaningful)
    'enq_purpose': 'nunique'

}).reset_index()

In [52]:
df_enquiry_agg.columns = [
    'customer_no',
    'total_enq_amt',
    'avg_enq_amt',
    'max_enq_amt',
    'first_enq_dt',
    'last_enq_dt',
    'total_enquiries',
    'unique_enq_purpose'
]

In [53]:
pd.set_option('display.max_columns', None)

In [54]:
query_demo = "SELECT * FROM Cust_Demographics"
df_demo = pd.read_sql(query_demo, conn)         ##getting data for 3rd sheet from database

print(df_demo.shape)
df_demo.head(2)

/tmp/ipython-input-332065695.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_demo = pd.read_sql(query_demo, conn)         ##getting data for 3rd sheet from database


(23896, 83)


,dt_opened,customer_no,entry_time,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,Bad_label
0,18-Apr-15,1,13-Apr-15,Insignia,13-Apr-15,650,2,Card Setup,14,500000,,,,Y,IS1,,0,159,4284,4284,,1,ADFPNXXXXX,03-Sep-65,98332XXXXX,N,@REDIFFMAIL.COM,1,2,,Mumbai / Navi Mumbai / Thane,400610,1965,0,Self,Y,2,90000,,,,0,0,0,0,Mumbai,400059,@CODOGNOTTO.NET,PAN Card,ADFPNXXXXX,The Ratnakar Bank Ltd.,,Y,State Bank of India,0,01-Jun-13,17-Jun-16,1,21,,Y,Y,N,,Y,1965-0,21,15,400610,0,2,90000,Nov-00,21,R,,,0000-00-00,0,98332XXXXX,1,N,0
1,21-Apr-15,2,21-Apr-15,Insignia,21-Apr-15,760,1,Card Setup,14,1200000,,,,Y,IS1,,0,91,B001,4077,,1,AJWPRXXXXX,14-Jul-62,99455XXXXX,N,@GMAIL.COM,1,2,,Bengaluru,560042,1969,0,Self,Y,2,1,,,,0,0,0,0,Bangalore,560042,,PAN Card,AJWPRXXXXX,The Ratnakar Bank Ltd.,,N,,0,,17-Jun-16,1,17,,Y,Y,N,,Y,1969-0,17,12,560042,0,2,1,Nov-00,17,R,,,0000-00-00,0,99455XXXXX,1,N,0


In [55]:
df_demo.columns

Index(['dt_opened', 'customer_no', 'entry_time', 'feature_1', 'feature_2',
       'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7',
       'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12',
       'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17',
       'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22',
       'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27',
       'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_32',
       'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_37',
       'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42',
       'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47',
       'feature_48', 'feature_49', 'feature_50', 'feature_51', 'feature_52',
       'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_57',
       'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62',
       '

In [56]:
demo_drop_columns=['feature_20','feature_22','feature_24','feature_45','feature_46','feature_47','feature_48','feature_51','feature_77','feature_10','feature_14',
                   'feature_15','feature_16','feature_17','feature_49','feature_53','feature_70','feature_74','feature_75']
##dropping columns which are having more than 50% missing values

In [57]:
df_demo=df_demo.drop(columns=demo_drop_columns, errors='ignore')

In [58]:
df_demo.columns

Index(['dt_opened', 'customer_no', 'entry_time', 'feature_1', 'feature_2',
       'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7',
       'feature_8', 'feature_9', 'feature_11', 'feature_12', 'feature_13',
       'feature_18', 'feature_19', 'feature_21', 'feature_23', 'feature_25',
       'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30',
       'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35',
       'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40',
       'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_50',
       'feature_52', 'feature_54', 'feature_55', 'feature_56', 'feature_57',
       'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62',
       'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_67',
       'feature_68', 'feature_69', 'feature_71', 'feature_72', 'feature_73',
       'feature_76', 'feature_78', 'feature_79', 'Bad_label'],
      dtype='object')

In [59]:
demo_date_columns=['dt_opened','entry_time','feature_2','feature_21','feature_54','feature_63']
demo_num_columns=['customer_no','feature_3','feature_4','feature_6','feature_7','feature_19','feature_25',
                  'feature_26','feature_29','feature_30','feature_31','feature_34','feature_35','feature_39','feature_40','feature_41','feature_42','feature_44',
                  'feature_52','feature_55','feature_56','feature_64','feature_65','feature_66','feature_67','feature_68','feature_69','feature_71',
                  'feature_76','feature_78']

In [60]:
for col in demo_date_columns:
    df_demo[col] = pd.to_datetime(df_demo[col], errors='coerce')

/tmp/ipython-input-741000090.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_demo[col] = pd.to_datetime(df_demo[col], errors='coerce')
/tmp/ipython-input-741000090.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_demo[col] = pd.to_datetime(df_demo[col], errors='coerce')
/tmp/ipython-input-741000090.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_demo[col] = pd.to_datetime(df_demo[col], errors='coerce')
/tmp/ipython-input-741000090.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure

In [61]:
for col in demo_num_columns:
    df_demo[col] = pd.to_numeric(df_demo[col], errors='coerce')

In [62]:
pd.set_option('display.max_rows', None)

In [63]:
df_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23896 entries, 0 to 23895
Data columns (total 64 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   dt_opened    23896 non-null  datetime64[ns]
 1   customer_no  23896 non-null  int64         
 2   entry_time   23881 non-null  datetime64[ns]
 3   feature_1    23896 non-null  object        
 4   feature_2    21060 non-null  datetime64[ns]
 5   feature_3    21060 non-null  float64       
 6   feature_4    23881 non-null  float64       
 7   feature_5    23896 non-null  object        
 8   feature_6    23881 non-null  float64       
 9   feature_7    23881 non-null  float64       
 10  feature_8    23896 non-null  object        
 11  feature_9    23896 non-null  object        
 12  feature_11   23896 non-null  object        
 13  feature_12   23896 non-null  object        
 14  feature_13   23896 non-null  object        
 15  feature_18   23896 non-null  object        
 16  feat

In [64]:
demo_obj_cols = df_demo.select_dtypes(include=['object', 'category'])
demo_obj_cols.head()

,feature_1,feature_5,feature_8,feature_9,feature_11,feature_12,feature_13,feature_18,feature_23,feature_27,feature_28,feature_32,feature_33,feature_36,feature_37,feature_38,feature_43,feature_50,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_72,feature_73,feature_79,Bad_label
0,Insignia,Card Setup,,,Y,IS1,,,N,,Mumbai / Navi Mumbai / Thane,Self,Y,,,,Mumbai,Y,,Y,Y,N,,Y,R,,N,0
1,Insignia,Card Setup,,,Y,IS1,,,N,,Bengaluru,Self,Y,,,,Bangalore,N,,Y,Y,N,,Y,R,,N,0
2,Insignia,Card Setup,,,Y,IS1,,,N,,Bengaluru,Self,Y,,,,Bangalore,N,,N,Y,N,,Y,R,,N,0
3,Insignia,Card Setup,,,Y,IS1,,,N,,Pune,Self,Y,,,,Pune,Y,,Y,Y,N,,Y,R,,N,0
4,Insignia,Card Setup,,,Y,IS1,,,N,,Gurgaon,Self,Y,,,,Gurgaon,N,,Y,Y,N,,Y,R,,N,0


In [65]:
len(demo_obj_cols.columns)

28

In [66]:
for col in demo_obj_cols:
    print(f"\nColumn: {col}")
    print(df_demo[col].value_counts())


Column: feature_1
feature_1
Platinum Maxima    9056
Platinum Deligh    8541
Titanium Deligh    5943
RBL Bank Fun+       218
Insignia             67
Golf Card            30
Platinum Cricke      26
                     15
Name: count, dtype: int64

Column: feature_5
feature_5
Card Setup    23881
                 15
Name: count, dtype: int64

Column: feature_8
feature_8
         22635
ROTVR      336
RCFO       275
RMTVR      249
RCFR       166
RRTVR      108
RNOI        94
RRCPV        8
RCBSR        7
ROCPV        4
RMIC         3
RPRN         2
RFCU         2
RCBNH        1
RNOA         1
RROCL        1
RINF         1
RMINV        1
RUTR         1
RCBL         1
Name: count, dtype: int64

Column: feature_9
feature_9
                                                             22635
Negative Office Tele Verification                              336
Contactability Failure at Office                               275
Negative Mobile tele Verification                              249
Contac

In [67]:
from sklearn.preprocessing import OrdinalEncoder

cat_cols = df_demo.select_dtypes(include='object').columns   # encoding for converting categorical column to numerical type for all 28 categorical columns

df_demo[cat_cols] = df_demo[cat_cols].fillna('Unknown')

 #Rare category grouping
threshold = 100

for col in cat_cols:
    freq = df_demo[col].value_counts()
    rare_labels = freq[freq < threshold].index
    df_demo[col] = df_demo[col].replace(rare_labels, 'Other')

#Encoding
encoder = OrdinalEncoder()
df_demo[cat_cols] = encoder.fit_transform(df_demo[cat_cols])

In [68]:
df_demo.duplicated().sum()

np.int64(0)

In [69]:
df_demo.dtypes

,0
dt_opened,datetime64[ns]
customer_no,int64
entry_time,datetime64[ns]
feature_1,float64
feature_2,datetime64[ns]
feature_3,float64
feature_4,float64
feature_5,float64
feature_6,float64
feature_7,float64


In [70]:
df_demo['customer_no'].duplicated().sum()

np.int64(0)

In [71]:
df_final = df_account_agg.merge(
                df_enquiry_agg,
                on='customer_no',
                how='left'
            ).merge(
                df_demo,
                on='customer_no',
                how='left'
            )                                               ##merging of all three dataset into dingle dataframe

In [72]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23896 entries, 0 to 23895
Data columns (total 94 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   customer_no          23896 non-null  int64         
 1   high_credit_amt      23896 non-null  float64       
 2   cur_balance_amt      23896 non-null  int64         
 3   amt_past_due         23896 non-null  float64       
 4   creditlimit          23896 non-null  float64       
 5   cashlimit            23896 non-null  float64       
 6   rateofinterest       12982 non-null  float64       
 7   actualpaymentamount  19207 non-null  float64       
 8   paymentfrequency     20932 non-null  float64       
 9   ph1_max_dpd          23896 non-null  int64         
 10  ph2_max_dpd          23896 non-null  int64         
 11  ph1_has_30_plus      23896 non-null  int64         
 12  ph2_has_30_plus      23896 non-null  int64         
 13  ph1_perc_good        23896 non-

In [73]:
df_final.isnull().sum()

,0
customer_no,0
high_credit_amt,0
cur_balance_amt,0
amt_past_due,0
creditlimit,0
cashlimit,0
rateofinterest,10914
actualpaymentamount,4689
paymentfrequency,2964
ph1_max_dpd,0


In [74]:
df_final['Bad_label'].value_counts() ##checking of target variable balance

,count
Bad_label,
0.0,22892
1.0,1004


In [75]:
df_final.isnull().sum()[df_final.isnull().sum() > 0]

,0
rateofinterest,10914
actualpaymentamount,4689
paymentfrequency,2964
closed_dt,6878
last_paymt_dt,51
avg_enq_amt,109
max_enq_amt,109
first_enq_dt,109
last_enq_dt,109
entry_time,15


In [76]:
null_percentage = (df_final.isnull().sum() / len(df_final)) * 100
null_percentage[null_percentage > 0]

,0
rateofinterest,45.672916
actualpaymentamount,19.622531
paymentfrequency,12.403750
closed_dt,28.783060
last_paymt_dt,0.213425
avg_enq_amt,0.456143
max_enq_amt,0.456143
first_enq_dt,0.456143
last_enq_dt,0.456143
entry_time,0.062772


In [77]:
df_final.drop(columns=['feature_63'], inplace=True)

# 2 Convert date columns to days difference                   ##converting date difference into number of days
reference_date = df_final['last_paymt_dt'].max()

date_cols = ['closed_dt','last_paymt_dt','first_enq_dt',
             'last_enq_dt','entry_time','feature_2',
             'feature_21','feature_54']

for col in date_cols:
    df_final[col + '_days'] = (reference_date - df_final[col]).dt.days

df_final.drop(columns=date_cols, inplace=True)

#  Median imputation for numeric
from sklearn.impute import SimpleImputer                                     ##replacing null values with median using simple imputer
num_cols = df_final.select_dtypes(include=['float64','int64']).columns
imputer = SimpleImputer(strategy='median')
df_final[num_cols] = imputer.fit_transform(df_final[num_cols])

In [78]:
df_final.isnull().sum()

,0
customer_no,0
high_credit_amt,0
cur_balance_amt,0
amt_past_due,0
creditlimit,0
cashlimit,0
rateofinterest,0
actualpaymentamount,0
paymentfrequency,0
ph1_max_dpd,0


In [79]:
from sklearn.model_selection import train_test_split      ##splitting dataframe into input and output variables
X = df_final.drop(['Bad_label','customer_no'], axis=1)
y = df_final['Bad_label']


In [80]:
X = X.select_dtypes(exclude=['datetime64[ns]'])

In [81]:
##splitting data into train ,test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [82]:
pd.set_option('display.max_rows',None)

In [83]:
X_train.dtypes

,0
high_credit_amt,float64
cur_balance_amt,float64
amt_past_due,float64
creditlimit,float64
cashlimit,float64
rateofinterest,float64
actualpaymentamount,float64
paymentfrequency,float64
ph1_max_dpd,float64
ph2_max_dpd,float64


In [84]:
from sklearn.linear_model import LogisticRegression   ##first attempting with logistic regression with balanced data for gini calculation

model_balanced = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)

model_balanced.fit(X_train, y_train)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

In [85]:
from sklearn.metrics import roc_auc_score, accuracy_score

y_pred_prob = model_balanced.predict_proba(X_test)[:,1]

auc = roc_auc_score(y_test, y_pred_prob)
gini = 2*auc - 1

print("AUC:", auc)
print("Gini:", gini)

AUC: 0.6254673346523553
Gini: 0.25093466930471053


In [86]:
from imblearn.over_sampling import SMOTE     ##using SMOTE to bring balance between 1 and 0 in the target variable

smote = SMOTE(random_state=42)

X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print(y_train_smote.value_counts())

Bad_label
0.0    18313
1.0    18313
Name: count, dtype: int64


In [87]:
model_smote = LogisticRegression(
    max_iter=1000,
    random_state=42
)

model_smote.fit(X_train_smote, y_train_smote)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=42)

In [88]:
y_pred_prob_smote = model_smote.predict_proba(X_test)[:,1]

auc_smote = roc_auc_score(y_test, y_pred_prob_smote)
gini_smote = 2*auc_smote - 1

print("SMOTE AUC:", auc_smote)
print("SMOTE Gini:", gini_smote)

SMOTE AUC: 0.6307532005836726
SMOTE Gini: 0.26150640116734514


In [89]:
df_final['utilization_ratio'] = df_final['cur_balance_amt'] / df_final['creditlimit']

df_final['cash_util_ratio'] = df_final['cur_balance_amt'] / df_final['cashlimit']

df_final['past_due_ratio'] = df_final['amt_past_due'] / df_final['cur_balance_amt']

df_final['payment_to_balance'] = df_final['actualpaymentamount'] / df_final['cur_balance_amt']

df_final['enquiry_intensity'] = df_final['total_enq_amt'] / df_final['total_enquiries']

In [90]:
import numpy as np    ##using normalisation to improve features

money_cols = [
    'high_credit_amt','cur_balance_amt','amt_past_due',
    'creditlimit','cashlimit','total_enq_amt',
    'avg_enq_amt','max_enq_amt'
]

for col in money_cols:
    df_final[col] = np.log1p(df_final[col])

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [91]:
df_final['max_dpd_combined'] = df_final[['ph1_max_dpd','ph2_max_dpd']].max(axis=1)    ##creating important features

df_final['ever_30_plus'] = df_final[['ph1_has_30_plus','ph2_has_30_plus']].max(axis=1)

df_final['avg_good_perc'] = (df_final['ph1_perc_good'] + df_final['ph2_perc_good']) / 2

In [92]:
drop_cols = [
    'customer_no',
    'opened_dt','reporting_dt','paymt_str_dt','paymt_end_dt','dt_opened',
    'Bad_label'  # keep separately
]

In [93]:
df_model = df_final.drop(columns=drop_cols)


In [94]:
y = df_final['Bad_label']
X = df_model

In [95]:
def woe_binning(df, feature, target, bins=10):          ##using Weight of evidence method to improve the model performance

    df = df[[feature, target]].copy()

    df['bin'] = pd.qcut(df[feature], q=bins, duplicates='drop')

    grouped = df.groupby('bin')[target].agg(['count','sum'])
    grouped.columns = ['total','bad']

    grouped['good'] = grouped['total'] - grouped['bad']

    total_good = grouped['good'].sum()
    total_bad = grouped['bad'].sum()

    grouped['dist_good'] = (grouped['good'] + 0.5) / total_good
    grouped['dist_bad'] = (grouped['bad'] + 0.5) / total_bad

    grouped['woe'] = np.log(grouped['dist_good'] / grouped['dist_bad'])

    grouped['iv'] = (grouped['dist_good'] - grouped['dist_bad']) * grouped['woe']

    return grouped

In [96]:
X_numeric = df_final.select_dtypes(include=[np.number])


In [97]:
X_numeric = X_numeric.drop(columns=['Bad_label'])

In [98]:
X_numeric = X_numeric.drop(columns=['customer_no'])

In [99]:
iv_list = []   ##Doing IV to identify the important features

for col in X_numeric.columns:
    try:
        table = woe_binning(df_final[[col, 'Bad_label']], col, 'Bad_label', bins=10)
        iv = table['iv'].sum()
        iv_list.append((col, iv))
    except Exception as e:
        print(f"Skipped {col}: {e}")

/tmp/ipython-input-1507482006.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby('bin')[target].agg(['count','sum'])
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:4779: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/tmp/ipython-input-1507482006.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby('bin')[target].agg(['count','sum'])
/tmp/ipython-input-1507482006.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to 

Skipped cur_balance_amt: missing values must be missing in the same location both left and right sides


/tmp/ipython-input-1507482006.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby('bin')[target].agg(['count','sum'])
/tmp/ipython-input-1507482006.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby('bin')[target].agg(['count','sum'])
/tmp/ipython-input-1507482006.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby('bin')[target].agg(['count','sum'])
/tmp/ip

Skipped utilization_ratio: missing values must be missing in the same location both left and right sides
Skipped cash_util_ratio: missing values must be missing in the same location both left and right sides
Skipped payment_to_balance: missing values must be missing in the same location both left and right sides


/tmp/ipython-input-1507482006.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby('bin')[target].agg(['count','sum'])
/tmp/ipython-input-1507482006.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby('bin')[target].agg(['count','sum'])
/tmp/ipython-input-1507482006.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby('bin')[target].agg(['count','sum'])
/tmp/ip

In [100]:
iv_df = pd.DataFrame(iv_list, columns=['Feature','IV']) ##the top 20 important features are below

iv_df = iv_df.sort_values(by='IV', ascending=False)

print(iv_df.head(20))

                Feature        IV
26            feature_7  0.163963
2           creditlimit  0.151389
3             cashlimit  0.135228
56           feature_52  0.120981
13      ph1_good_months  0.115820
14      ph2_good_months  0.086891
12        ph2_perc_good  0.075275
80     last_enq_dt_days  0.063341
22            feature_3  0.057395
79    first_enq_dt_days  0.054040
15            acct_type  0.050285
89        avg_good_perc  0.040149
5   actualpaymentamount  0.035943
83      feature_21_days  0.034914
82       feature_2_days  0.031979
30           feature_12  0.031610
78   last_paymt_dt_days  0.030936
81      entry_time_days  0.030152
0       high_credit_amt  0.028057
21            feature_1  0.027167


In [101]:
iv_df.head(10)

,Feature,IV
26,feature_7,0.163963
2,creditlimit,0.151389
3,cashlimit,0.135228
56,feature_52,0.120981
13,ph1_good_months,0.115820
14,ph2_good_months,0.086891
12,ph2_perc_good,0.075275
80,last_enq_dt_days,0.063341
22,feature_3,0.057395
79,first_enq_dt_days,0.054040


In [102]:
selected_features = iv_df[iv_df['IV'] > 0.05]['Feature'].tolist()

In [103]:
corr = df_final[selected_features].corr()
print(corr)

                   feature_7  creditlimit  cashlimit  feature_52  \
feature_7           1.000000     0.205902   0.211404    0.573739   
creditlimit         0.205902     1.000000   0.837714    0.231927   
cashlimit           0.211404     0.837714   1.000000    0.238993   
feature_52          0.573739     0.231927   0.238993    1.000000   
ph1_good_months     0.252049     0.357270   0.358257    0.268564   
ph2_good_months     0.311074     0.293246   0.301339    0.253547   
ph2_perc_good       0.178844     0.117991   0.116943    0.146921   
last_enq_dt_days    0.126104    -0.006241   0.012707    0.091322   
feature_3           0.300612    -0.096530  -0.076081    0.056958   
first_enq_dt_days   0.289401     0.197113   0.193241    0.181870   
acct_type           0.235728     0.320169   0.316847    0.218259   

                   ph1_good_months  ph2_good_months  ph2_perc_good  \
feature_7                 0.252049         0.311074       0.178844   
creditlimit               0.357270         

In [104]:
df_final['limit_util_interaction'] = df_final['creditlimit'] * df_final['utilization_ratio']

In [105]:
df_final['good_month_ratio'] = df_final['ph1_good_months'] / (df_final['ph1_good_months']+ df_final['ph2_good_months'] + 1)

In [106]:
model = LogisticRegression(max_iter=1000, class_weight='balanced')

In [107]:
print(df_final['Bad_label'].mean())

0.04201540006695681


In [108]:
selected_features = iv_df[iv_df['IV'] > 0.03]['Feature'].tolist() ##selecting the strong features
print("Number of selected features:", len(selected_features))

Number of selected features: 18


In [109]:
df_final['dpd_trend'] = df_final['ph2_max_dpd'] - df_final['ph1_max_dpd']
df_final['good_month_ratio'] = df_final['ph1_good_months'] / (df_final['ph1_good_months'] + 1)
df_final['limit_util'] = df_final['cur_balance_amt'] / (df_final['creditlimit'] + 1)

/tmp/ipython-input-4076459356.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['dpd_trend'] = df_final['ph2_max_dpd'] - df_final['ph1_max_dpd']
/tmp/ipython-input-4076459356.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['limit_util'] = df_final['cur_balance_amt'] / (df_final['creditlimit'] + 1)


In [110]:
X = df_final[selected_features]
y = df_final['Bad_label']

In [111]:
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)

In [112]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

In [113]:
from sklearn.linear_model import LogisticRegression   ##modelling with WOE filtered data

model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',   # important for 4% bad rate
    solver='lbfgs'
)

model.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', max_iter=1000)

In [114]:
from sklearn.metrics import roc_auc_score

train_pred = model.predict_proba(X_train)[:,1]
test_pred  = model.predict_proba(X_test)[:,1]

In [115]:
train_auc = roc_auc_score(y_train, train_pred)
test_auc  = roc_auc_score(y_test, test_pred)

print("Train AUC:", train_auc)
print("Test AUC :", test_auc)

Train AUC: 0.6206294221541088
Test AUC : 0.6336140258544126


In [116]:
train_gini = 2 * train_auc - 1
test_gini  = 2 * test_auc - 1   ##prediction score with WOE data

print("Train Gini:", round(train_gini, 4))
print("Test Gini :", round(test_gini, 4))

Train Gini: 0.2413
Test Gini : 0.2672


In [117]:
print("Gini Gap (Train - Test):", round(train_gini - test_gini, 4))

Gini Gap (Train - Test): -0.026


In [118]:
from scipy.stats import ks_2samp

ks_stat = ks_2samp(
    test_pred[y_test==0],
    test_pred[y_test==1]
).statistic

print("Test KS:", round(ks_stat,4))

Test KS: 0.2205


In [119]:
import numpy as np

# Use IV > 0.03 features
selected_features = iv_df[iv_df['IV'] > 0.03]['Feature'].tolist()

X = df_final[selected_features]
y = df_final['Bad_label']

# Safety cleaning
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)

In [120]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=6,
    min_samples_leaf=50,
    random_state=42,
    class_weight='balanced'
)

rf.fit(X_train, y_train)

rf_train_pred = rf.predict_proba(X_train)[:,1]
rf_test_pred  = rf.predict_proba(X_test)[:,1]

rf_train_auc = roc_auc_score(y_train, rf_train_pred)
rf_test_auc  = roc_auc_score(y_test, rf_test_pred)
rf_train_gini = 2*rf_train_auc - 1
rf_test_gini  = 2*rf_test_auc - 1

print("Random Forest Train Gini:", round(rf_train_gini,4))
print("Random Forest Test Gini :", round(rf_test_gini,4))

Random Forest Train Gini: 0.5839
Random Forest Test Gini : 0.3092


In [121]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

gb.fit(X_train, y_train)

gb_train_pred = gb.predict_proba(X_train)[:,1]
gb_test_pred  = gb.predict_proba(X_test)[:,1]

gb_train_auc = roc_auc_score(y_train, gb_train_pred)
gb_test_auc  = roc_auc_score(y_test, gb_test_pred)
gb_train_gini = 2*gb_train_auc - 1
gb_test_gini  = 2*gb_test_auc - 1

print("Gradient Boosting Train Gini:", round(gb_train_gini,4))
print("Gradient Boosting Test Gini :", round(gb_test_gini,4))

Gradient Boosting Train Gini: 0.6867
Gradient Boosting Test Gini : 0.2807


In [122]:
feature_importance = pd.DataFrame({
    'Feature': selected_features,
    'Importance': rf.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(feature_importance.head(10))

               Feature  Importance
0            feature_7    0.121750
1          creditlimit    0.089199
2            cashlimit    0.087192
8            feature_3    0.077100
7     last_enq_dt_days    0.074044
3           feature_52    0.063119
4      ph1_good_months    0.057106
6        ph2_perc_good    0.051238
5      ph2_good_months    0.049530
16  last_paymt_dt_days    0.047698
